In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
good_data = pd.read_csv("../../data/system_prompts/Prompt-Scores_Good-Property.csv")
bad_data = pd.read_csv("../../data/system_prompts/Prompt-Scores_Bad-Property.csv")
full_data = pd.concat([good_data, bad_data], ignore_index=True)
full_data

,Prompt,Catagory,Comments,llama-2-7b-chat/ARC_acc,llama-2-7b-chat/ARC_acc_no_error,llama-2-7b-chat/ARC_error,llama-2-7b-chat/MMLU_acc,llama-2-7b-chat/MMLU_acc_no_error,llama-2-7b-chat/MMLU_error,llama-2-7b-chat/HELLASWAG_acc,...,Qwen1.5-0.5B-Chat/MMLU_error,Qwen1.5-0.5B-Chat/HELLASWAG_acc,Qwen1.5-0.5B-Chat/HELLASWAG_acc_no_error,Qwen1.5-0.5B-Chat/HELLASWAG_error,Mistral-7B-Instruct-v0.2/MMLU_acc,Mistral-7B-Instruct-v0.2/MMLU_acc_no_error,Mistral-7B-Instruct-v0.2/MMLU_error,Jamba-v0.1/ARC_acc,Jamba-v0.1/ARC_acc_no_error,Jamba-v0.1/ARC_error
0,"You are a contextual assistant, able to unders...",Good-Property,PropertyCount: 1,0.516212,0.516212,0,0.408221,0.408554,2,0.494,...,19,0.265,0.265797,3,0.543346,0.548030,21,0.656143,0.696558,68.0
1,"You are an assertive assistant, confidently ta...",Good-Property,PropertyCount: 1,0.512799,0.512799,0,0.409035,0.409202,1,0.487,...,18,0.275,0.278340,12,0.529101,0.533881,22,0.662116,0.684303,38.0
2,"You are an organized assistant, capable of kee...",Good-Property,PropertyCount: 1,0.515358,0.515358,0,0.404558,0.404888,2,0.463,...,24,0.232,0.233635,7,0.536020,0.541085,23,0.696246,0.717678,35.0
3,"You are a balanced assistant, able to juggle m...",Good-Property,PropertyCount: 1,0.505973,0.505973,0,0.402930,0.403094,1,0.480,...,18,0.268,0.268537,2,0.538462,0.542658,19,0.683447,0.703866,34.0
4,"You are a robust assistant, capable of handlin...",Good-Property,PropertyCount: 1,0.515358,0.515358,0,0.404965,0.405130,1,0.474,...,15,0.311,0.312249,4,0.538462,0.543550,23,0.683447,0.713268,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
605,"You are an unnecessary, indifferent, distracti...",Bad-Property,PropertyCount: 30,0.473549,0.473549,0,0.395604,0.395765,1,0.422,...,15,0.260,0.262626,10,0.518926,0.527732,41,NaN,NaN,NaN
606,"You are a provocative, superficial, speculativ...",Bad-Property,PropertyCount: 30,0.474403,0.474403,0,0.403337,0.403995,4,0.406,...,22,0.276,0.278788,10,0.516891,0.524577,36,NaN,NaN,NaN
607,"You are an energy-sapping, heartless, startlin...",Bad-Property,PropertyCount: 30,0.435154,0.435154,0,0.384615,0.385243,4,0.380,...,21,0.267,0.268882,7,0.522996,0.529460,30,NaN,NaN,NaN
608,"You are an implausible, deceptive, dubious, tr...",Bad-Property,PropertyCount: 30,0.459898,0.459898,0,0.404965,0.405460,3,0.389,...,13,0.262,0.263581,6,0.518519,0.529070,49,NaN,NaN,NaN


In [4]:
f = open("../../data/prompt_generation/good_property.txt", "r")
good_properties = f.read().split(", ")
f.close()

f = open("../../data/prompt_generation/bad_property.txt", "r")
bad_properties = f.read().split(", ")
f.close()

all_properties = good_properties + bad_properties

In [16]:
benchmark_cols = []
for name in list(full_data.columns):
    if "_acc_no_error" in name and "Jamba":
        benchmark_cols.append(name)
benchmark_cols

df_reg = pd.melt(full_data, id_vars=['Prompt'], value_vars=benchmark_cols, value_name='Accuracy').dropna()
df_reg["Model_Name"] = df_reg["variable"].apply(lambda x: x.split("/")[0])
df_reg["Benchmark_Name"] = df_reg["variable"].apply(lambda x: x.split("/")[1].split("_")[0])
df_reg = df_reg.drop(columns=["variable"])

for one_property in all_properties:
    df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)

df_reg

C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is

C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is

C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is

C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is

C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is

...

C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is

C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is

C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is

C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_reg[one_property] = df_reg["Prompt"].apply(lambda x: 1 if one_property.lower() in x.lower() else 0)
C:\Users\zhang\AppData\Local\Temp\ipykernel_11212\1540438995.py:13: PerformanceWarning: DataFrame is

,Prompt,Accuracy,Model_Name,Benchmark_Name,Forward-thinking,Intelligent,Proficient,Superior,Prime,Streamlined,...,Unfounded,Clunky,Hopeless,Illogical,Heartless,Distressing,Patronizing,Intrusive,Aggravating,Deflating
0,"You are a contextual assistant, able to unders...",0.516212,llama-2-7b-chat,ARC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"You are an assertive assistant, confidently ta...",0.512799,llama-2-7b-chat,ARC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"You are an organized assistant, capable of kee...",0.515358,llama-2-7b-chat,ARC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"You are a balanced assistant, able to juggle m...",0.505973,llama-2-7b-chat,ARC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"You are a robust assistant, capable of handlin...",0.515358,llama-2-7b-chat,ARC,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14940,"You are an anomaly-detecting, exemplary, analy...",0.683124,Jamba-v0.1,ARC,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14941,"You are a trailblazing, reliable, versatile, p...",0.675462,Jamba-v0.1,ARC,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14942,"You are an AI system that embodies progress, i...",0.645051,Jamba-v0.1,ARC,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14943,"You are a streamlined, state-of-the-art, world...",0.689201,Jamba-v0.1,ARC,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [17]:
df_reg.to_csv("../../results/system_prompts/Prompt-Scores_for_Regression.csv", index=False)